In [1]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib as plt

In [2]:
from NRI.experiments.config import ExperimentConfig


experiment_config = ExperimentConfig(
    obs_len=12,
    pred_len=12,
    encoder_loss_weight=1.0,
    decoder_loss_weight=1.0,
    signal_loss_weight=1.0,
    n_epoch=20,
    batch_size=64,
    checkpoint_prefix='dnri_12_NoTraff',
    checkpoint_interval=10,
    use_cuda=True,
    mask_traffic_signal=True
)

In [3]:
from SinD.config import get_dataset_path
from SinD.dataset.io import get_dataset_records
from NRI.dataset.utils import split_dataset

dataset_path = get_dataset_path()
dataset_files = get_dataset_records(dataset_path)
train_files, valid_files, test_files = split_dataset(dataset_files, 0.7, 0.2)

load datasets

In [4]:
from NRI.dataset import SignalizedIntersectionDatasetForNRI, SignalizedIntersectionDatasetConfig

dataset_config = SignalizedIntersectionDatasetConfig(
    obs_len=experiment_config.obs_len,
    pred_len=experiment_config.pred_len,
    stride=15,
    encode_traffic_signals=True,
    padding_value=0.0
)

train_set = SignalizedIntersectionDatasetForNRI(dataset_config)
train_set.load_records(dataset_path, train_files, verbose=True)

valid_set = SignalizedIntersectionDatasetForNRI(dataset_config)
valid_set.load_records(dataset_path, valid_files, verbose=True)


load_records: 100%|██████████| 4/4 [00:35<00:00,  8.84s/it]


In [5]:
from NRI.models import DynamicNeuralRelationalInference
from NRI.experiments.main import train

model = DynamicNeuralRelationalInference(
    hid_dim=64,
    n_edges=4,
    dgvae=False
)

if experiment_config.use_cuda:
    model.cuda()

optimizer = torch.optim.Adam(model.parameters())

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

train(
    model,
    train_set=train_set,
    valid_set=valid_set,
    config=experiment_config,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler
)

[train] epoch 0:   0%|          | 0/174 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: T

Overall Loss: 23.142181308790192
Encoder KL Loss: 0.018519533337789714
Decoder NLL Loss: 23.12366173185152
Signal Cross-Entropy Loss: 1.5064185741304013


[valid] epoch 0:   0%|          | 0/49 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: Th

Signal Prediction Accuracy: 0.0
Final Displacement Error: 6.328040911226856
Average Displacement Error: 3.118809262100531


[train] epoch 1:   0%|          | 0/174 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: T

...

Signal Prediction Accuracy: 0.0
Final Displacement Error: 5.245896120460666
Average Displacement Error: 2.4919705780185


[train] epoch 19:   0%|          | 0/174 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: 

Overall Loss: -9.009849474347869
Encoder KL Loss: 0.06421803334064186
Decoder NLL Loss: -9.07406749396489
Signal Cross-Entropy Loss: 1.5063890218734692


[valid] epoch 19:   0%|          | 0/49 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: T

Signal Prediction Accuracy: 0.0
Final Displacement Error: 5.243369944241582
Average Displacement Error: 2.487721010130279


In [3]:
import os
import torch
from torch.utils.data import DataLoader

from NRI.dataset import SignalizedIntersectionDatasetForNRI
from NRI.models import DynamicNeuralRelationalInference
from NRI.experiments.config import ExperimentConfig
from NRI.experiments.test import generate_result
from SinD.config import get_dataset_path
from SinD.dataset.io import get_dataset_records
from NRI.dataset.utils import split_dataset
from NRI.dataset import SignalizedIntersectionDatasetForNRI, SignalizedIntersectionDatasetConfig

experiment_config = ExperimentConfig(
    obs_len=12,
    pred_len=12,
    encoder_loss_weight=1.0,
    decoder_loss_weight=1.0,
    signal_loss_weight=1.0,
    n_epoch=20,
    batch_size=64,
    checkpoint_prefix='dnri_12_NoTraff',
    checkpoint_interval=10,
    use_cuda=True,
    mask_traffic_signal=True
)

# Load the dataset
dataset_path = get_dataset_path()
dataset_files = get_dataset_records(dataset_path)
_, _, test_files = split_dataset(dataset_files, 0.7, 0.2)

dataset_config = SignalizedIntersectionDatasetConfig(
    obs_len=experiment_config.obs_len,
    pred_len=experiment_config.pred_len,
    stride=15,
    encode_traffic_signals=True,
    padding_value=0.0,
    include_incomplete_trajectories=False
)

test_set = SignalizedIntersectionDatasetForNRI(dataset_config)
test_set.load_records(dataset_path, test_files, verbose=True)

model = DynamicNeuralRelationalInference(
    hid_dim=64,
    n_edges=4,
    dgvae=False
)

# Load the saved model and configuration
checkpoint = torch.load('../checkpoints/dnri_12_NoTraff_best.pt')
model.load_state_dict(checkpoint['params'])
test_stats = generate_result(model, test_set, experiment_config)

# Report the results
test_stats.report()

[test] generating result: 100%|██████████| 34/34 [00:06<00:00,  5.55it/s]

=== Overall Result ===
Signal Prediction Accuracy: 0.0
Overall Final Displacement Error: 2.4803903102874756
Overall Average Displacement Error: 1.0032001733779907
=== Displacement Error of Class `pedestrian` ===
FDE: 5.000961668067253
ADE: 2.6214960854557843
=== Displacement Error of Class `car` ===
FDE: 2.4817559997956034
ADE: 0.704604554504484
=== Displacement Error of Class `truck` ===
FDE: 21.160104630454892
ADE: 5.962149904620263
=== Displacement Error of Class `bus` ===
FDE: 0.5421742046517986
ADE: 0.24542376993049264
=== Displacement Error of Class `motorcycle` ===
FDE: 2.5717481467376886
ADE: 1.0075953586331956
=== Displacement Error of Class `tricycle` ===
FDE: 1.2659166227550704
ADE: 0.5014884084203437
=== Displacement Error of Class `bicycle` ===
FDE: 3.7331809061536547
ADE: 1.558264241219535
=== Overall Result ===
Signal Prediction Accuracy: 0.0
Overall Final Displacement Error: 2.4803903102874756
Overall Average Displacement Error: 1.0032001733779907
=== Displacement Error